# BigQuery - BQML

This notebook uses BigQuery ML to train a model to predict the value of handwritten digits in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then evaluated and used for prediction using BigQuery directly.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-02.png">

---
## Setup

Parameters:

In [1]:
PROJECT_ID = "statmike-mlops"
REGION = 'us-central1'

---
## Train Model

Use BigQuery ML to train multiclass logistic regression model:

In [2]:
%%bigquery
CREATE OR REPLACE MODEL `digits.digits_lr`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['target']
  ) AS
SELECT * EXCEPT(splits,target_OE)
FROM `digits.digits_prepped`
WHERE splits = 'TRAIN'

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1852.06query/s]                        


""


Review the iterations from training:

In [3]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `digits.digits_lr`)
ORDER BY iteration

Downloading: 100%|██████████| 14/14 [00:01<00:00, 12.87rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.194639,0.194364,0.2,5689
1,0,1,0.141403,0.141121,0.4,18363
2,0,2,0.086962,0.087841,0.8,8996
3,0,3,0.051978,0.054850,1.6,14015
4,0,4,0.032789,0.037127,3.2,9792
5,0,5,0.021810,0.026129,6.4,12609
6,0,6,0.016038,0.021299,12.8,6249
7,0,7,0.014091,0.019476,3.2,9814
8,0,8,0.012840,0.018503,6.4,11023
9,0,9,0.011479,0.017702,12.8,11866


---
## Evaluate Model

Review the model evaluation statistics on the Test/Train splits:

In [4]:
%%bigquery
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TRAIN'))
UNION ALL
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TEST'))

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.75rows/s]


,SPLIT,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,TRAIN,0.983329,0.983552,0.983460,0.983381,0.091191,0.999803
1,TEST,0.964503,0.963677,0.965318,0.963749,0.122869,0.999464


Review the confusion matrix for each split:

In [5]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TRAIN')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.60rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,141,0,0,0,0,0,0,0,0,0
1,1,0,145,0,0,0,0,0,0,2,4
2,2,0,0,146,0,0,0,0,0,0,0
3,3,0,0,0,155,0,0,0,0,3,0
4,4,0,0,0,0,148,0,0,0,1,0
5,5,0,0,0,0,0,134,1,0,0,2
6,6,0,1,0,0,1,0,146,0,0,0
7,7,0,0,0,0,0,0,0,140,0,1
8,8,0,4,1,0,0,2,0,0,135,0
9,9,0,0,0,0,0,1,0,0,0,137


In [6]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TEST')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.51rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,37,0,0,0,0,0,0,0,0,0
1,1,0,28,0,1,0,0,0,0,1,1
2,2,0,0,31,0,0,0,0,0,0,0
3,3,0,0,1,23,0,0,0,1,0,0
4,4,0,0,0,0,31,0,0,1,0,0
5,5,0,0,0,0,0,44,0,0,0,1
6,6,0,0,0,0,0,0,32,0,1,0
7,7,0,0,0,0,0,0,0,38,0,0
8,8,0,1,0,0,0,0,0,0,31,0
9,9,0,0,0,1,0,1,0,0,1,39


---
## Predictions

Create a pandas dataframe with predictions for the test data in the table:

In [7]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits='TEST')
  )

Downloading: 100%|██████████| 346/346 [00:01<00:00, 193.45rows/s]


Review columns from the predictions - note that the query added columns with prefix `predicted_`

In [8]:
pred.columns

Index(['predicted_target', 'predicted_target_probs', 'p0', 'p1', 'p2', 'p3',
       'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14',
       'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24',
       'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34',
       'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44',
       'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54',
       'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target',
       'target_OE', 'SPLITS'],
      dtype='object')

Print the first few rows for the columns related to the actual and predicted values:

In [9]:
pred[['target', 'predicted_target', 'predicted_target_probs', 'SPLITS']].head()

,target,predicted_target,predicted_target_probs,SPLITS
0,5,5,"[{'label': 5, 'prob': 0.995864246436268}, {'la...",TEST
1,5,5,"[{'label': 5, 'prob': 0.9975593123273012}, {'l...",TEST
2,1,1,"[{'label': 1, 'prob': 0.996760113108158}, {'la...",TEST
3,1,1,"[{'label': 1, 'prob': 0.995214070530646}, {'la...",TEST
4,8,8,"[{'label': 8, 'prob': 0.7058836861367389}, {'l...",TEST


Notice the nested dictionary for predicted probabilities.  In BigQuery this is a Record type structure with nested fields for `label` and `prop`.  This is returned to the pandas dataframe as a nested dictionary.

The following code sorts the dictionary for the first record by `prop`:

In [10]:
[sorted(x, key = lambda x: x['label']) for x in [pred.predicted_target_probs[0]]]

[[{'label': 0, 'prob': 3.503006785586101e-07},
  {'label': 1, 'prob': 6.4599759407259584e-06},
  {'label': 2, 'prob': 1.1426240877358945e-05},
  {'label': 3, 'prob': 0.0040147772074752555},
  {'label': 4, 'prob': 2.8075070564662266e-07},
  {'label': 5, 'prob': 0.995864246436268},
  {'label': 6, 'prob': 6.178662254459178e-07},
  {'label': 7, 'prob': 6.0253679607391576e-05},
  {'label': 8, 'prob': 1.5443286253411316e-05},
  {'label': 9, 'prob': 2.6144255968225256e-05}]]

---
## Remove Resources
see notebook "XX - Cleanup"